<div style="
  padding: 5pt;
  border-style: solid;
  border-width: 1px;
  border-color: gray;
  border-radius: 10px;">
  
# **Python et intelligence artificielle**

# *Complément au mécanisme de la rétropropagation*

</div>

Dans ce complément aux bloc-notes de la séance n°11, nous souhaitons vous amener à mieux comprendre le mécanisme de rétropropagation en l'appliquant à un unique neurone. Nous allons réaliser quelques étapes en utilisant les concepts théoriques de base, comme la dérivée d'une fonction et la règle de dérivation en chaîne, afin de valider les calculs des gradients et vérifier la modification des poids. 

Rappelons tout d'abord l'implémentation de la classe `Neurone` :

<div style="
    padding: 5pt;
    border-style: dashed;
    border-width: 1px;
    border-color: gray;">

```python
class Neurone:
    def __init__(self, nbr_entrees, fonction_activation='sigmoïde'):
        # Initialisation des poids et du biais
        self.poids = np.random.uniform(-0.1, 0.1, nbr_entrees)
        self.biais = np.random.normal(0, 1e-3)
        # Définition de la fonction d'activation
        self.fonction_activation = fonction_activation
        
    def _appliquer_activation(self, x):
        """Applique la fonction d'activation sélectionnée."""
        if self.fonction_activation == 'sigmoïde':
            return 1 / (1 + np.exp(-x))
        elif self.fonction_activation == 'relu':
            return np.maximum(0, x)
        elif self.fonction_activation == 'tanh':
            return np.tanh(x)
        else:
            raise ValueError("Fonction d'activation non reconnue.")

    def calculer_sortie(self, entrees):
        """ Calcule la sortie du neurone """
        somme = np.dot(entrees, self.poids) + self.biais
        return self._appliquer_activation(somme)
```

</div>

### 1. **Dérivée de la fonction d'activation**

Le calcul de la dérivée de la fonction d'activation est utilisée pour la rétropropagation car elle sert à déterminer comment ajuster les poids afin de réduire l'erreur du modèle. La dérivée nous indique comment la sortie change pour une variation infime de l'entrée, ce qui est nécessaire pour savoir comment ajuster chaque poids durant l'apprentissage.

Prenons la fonction **sigmoïde** :

$$
\phi(z) = \frac{1}{1 + e^{-z}}
$$

Cette fonction est couramment utilisée pour les neurones de sortie car elle produit des valeurs entre 0 et 1 ce qui est adapté à des problèmes de classification. Sa dérivée qui mesure la "pente" ou la "sensibilité" de cette fonction pour une valeur donnée de $z$, est féfinie par :

$$
\phi'(z) = \phi(z) \cdot (1 - \phi(z))
$$

Vérifions que notre dérivée analytique de la sigmoïde est correcte en utilisant le **quotient de différence symétrique** qui permet d'approcher la dérivée d'une fonction en observant les changements de cette fonction pour une petite variation de son argument. Ce quotient pour approximer $\phi'(z)$ est calculé de la manière suivante :

$$
\phi'(z) \approx \frac{\phi(z + h) - \phi(z - h)}{2h}
$$

où $h$ est un très petit nombre (par exemple, $10^{-5}$). Cette formule revient à calculer la pente de la ligne reliant les points $(z + h, \phi(z + h))$ et $(z - h, \phi(z - h))$. Cette "pente" approche la dérivée si $h$ est suffisamment petit.

Cette vérification sert uniquement à confirmer que la dérivée analytique $\phi'(z) = \phi(z) \cdot (1 - \phi(z))$ est correcte. Les valeurs devraient être très proches si notre implémentation est correcte. Vérifiez-le pour la valeur $z = 0.5$.

#### Solution


In [ ]:
# Votre code ici



#### Sortie

<div style="
    padding: 5pt;
    border-style: solid;
    border-width: 1px;
    border-color: lightgray;">

```python
Dérivée analytique : 0.2350037122015945
Dérivée numérique : 0.2350037122067494
```

</div>

## 2. **Calcul du gradient pour une seule sortie**

Pour un neurone unique, le gradient de l'erreur par rapport aux poids peut être calculé en appliquant
la [règle de dérivation en chaîne](https://fr.wikipedia.org/wiki/Théorème_de_dérivation_des_fonctions_composées). 
Ce gradient va nous indiquer dans quelle mesure chaque poids contribue à l'erreur totale. 

Rappelons que pour un neurone produisant une sortie $y$ avec une cible $y_{\text{cible}}$, l'erreur quadratique est donnée par la fonction de coût :

$$
J = \frac{1}{2}(y_{\text{cible}} - y)^2
$$

Pour ajuster les poids, nous devons savoir comment $J$ varie par rapport à chaque poids $w_i$, c'est-à-dire que nous devons calculer $\frac{\partial J}{\partial w_i}$ en appliquant la règle de dérivation en chaîne. Cela se fait en deux étapes :

1. Calculons $\delta$, l'écart entre la sortie $y$ et la cible $y_{\text{cible}}$ :

   $$
   \delta = y - y_{\text{cible}}
   $$

   Cela nous indique de combien le neurone s'éloigne de la cible.

2. En utilisant la règle de dérivation en chaîne, nous obtenons pour le gradient de $J$ en fonction des poids :

   $$
   \frac{\partial J}{\partial w_i} = \delta \cdot \phi'(z) \cdot x_i
   $$

   avec :

   - $\delta$ mesure l'erreur de sortie ;
   - $\phi'(z)$ représente la sensibilité de la fonction d'activation aux changements de $z$ ;
   - $x_i$ est l'entrée associée au poids $w_i$.

### Exemple

Prenons un neurone avec deux entrées initialisées respectivement à $0,6$ et $0,1$, et une sortie cible fixée à $0,8$. En comparant le gradient analytique et celui produit par votre classe, vérifiez la cohérence des calculs à $10^{-5}$ près.

#### Solution


In [ ]:
# Votre code ici



#### Sortie

<div style="
    padding: 5pt;
    border-style: solid;
    border-width: 1px;
    border-color: lightgray;">

```python
Gradient analytique : [-0.05987721 -0.00997953]
Gradient pour la classe : [-0.05987721 -0.00997953]
```

</div>

## 3. **Rétropropagation avec des valeurs simples**

Nous allons utiliser des valeurs simples pour les poids, les biais, les entrées, et la sortie cible, de sorte que vous puissiez calculer les résultats attendus manuellement.

### Exemple

1. Pour un neurone à deux entrées, fixez les poids à $0.5$ et $-0.5$ pour chacune des entrées et mettez le biais à $0.0$.
2. Initialisez les valeurs des deux entrées à $1.0$, la sortie cible à $1.0$ et le taux d'apprentissage à $0.1$.
3. Effectuez la rétropropagation et vérifiez que les valeurs mises à jour des poids et biais correspondent aux résultats que vous aurez calculés manuellement.

### Solution


In [ ]:
# Votre code ici


#### Sortie

<div style="
    padding: 5pt;
    border-style: solid;
    border-width: 1px;
    border-color: lightgray;">

```python
Mise à jour attendue des poids : [ 0.5125 -0.4875]
Poids après rétropropagation : [ 0.5125 -0.4875]
```

</div>

---

## Détail sur l'obtention de la dérivée partielle de la fonction de coût par rapport au poids

<div style="
  padding: 5pt;
  background-color: lightgray;
  border-style: solid;
  border-width: 2px;
  border-color: khaki;">

L'expression

$$
\frac{\partial J}{\partial w_i} = \delta \cdot \phi'(z) \cdot x_i
$$

provient de l'application de la règle de dérivation en chaîne qui permet d'obtenir la dérivée de la fonction de coût $J$ par rapport au poids $w_i$ d'un neurone.

Considérons que le neurone produit une sortie $y$, calculée en appliquant une fonction d'activation $\phi$ à une combinaison linéaire des entrées pondérées. La sortie cible est $y_{\text{cible}}$, et la fonction de coût est ici l'erreur quadratique moyenne :

$$
J = \frac{1}{2} (y_{\text{cible}} - y)^2
$$

où $y = \phi(z)$ et $z$ est la somme pondérée des entrées, soit :

$$
z = \sum_{i} w_i x_i + b
$$

Nous voulons connaître l'impact d'un changement du poids $w_i$ sur la fonction de coût $J$, c'est-à-dire calculer la dérivée partielle $\frac{\partial J}{\partial w_i}$. Pour ce faire, nous appliquons la règle de dérivation en chaîne, en décomposant $\frac{\partial J}{\partial w_i}$ en plusieurs étapes.

Tout d'abord, calculons la dérivée de $J$ par rapport à la sortie $y$ :

$$
\frac{\partial J}{\partial y} = \frac{\partial}{\partial y} \left( \frac{1}{2} (y_{\text{cible}} - y)^2 \right) = -(y_{\text{cible}} - y) = -\delta
$$

où $\delta = y - y_{\text{cible}}$ est l'erreur de prédiction du neurone.

Puisque $y = \phi(z)$, la dérivée de $y$ par rapport à $z$ est la dérivée de la fonction d'activation :

$$
\frac{\partial y}{\partial z} = \phi'(z)
$$

Ainsi, pour propager l'erreur $\delta$ vers $z$, nous devons multiplier par $\phi'(z)$.

Enfin, $z$ dépend de chaque poids $w_i$ et de l'entrée $x_i$ selon la relation $z = \sum_{i} w_i x_i + b$. La dérivée de $z$ par rapport à $w_i$ est simplement :

$$
\frac{\partial z}{\partial w_i} = x_i
$$

En appliquant la règle de dérivation en chaîne, nous obtenons :

$$
\frac{\partial J}{\partial w_i} = \frac{\partial J}{\partial y} \cdot \frac{\partial y}{\partial z} \cdot \frac{\partial z}{\partial w_i}
$$

Substituons les dérivées obtenues par leur valeur :

- $\frac{\partial J}{\partial y} = -\delta$,
- $\frac{\partial y}{\partial z} = \phi'(z)$,
- $\frac{\partial z}{\partial w_i} = x_i$.

Ce qui nous donne au final :

$$
\frac{\partial J}{\partial w_i} = -\delta \cdot \phi'(z) \cdot x_i
$$

Puisque $-\delta$ est simplement l'opposée de l'erreur $\delta$, on écrit finalement :

$$
\frac{\partial J}{\partial w_i} = \delta \cdot \phi'(z) \cdot x_i
$$

</div>

---

[Séance n°11a](seance_11a.ipynb) / [Séance n°11c](seance_11c.ipynb)